In [1]:
import numpy as np
import pandas as pd
import pycountry

### TRANSFORMACIONES ARCHIVOS MIGRACION NETA E INDICADORES BANCO MUNDIAL

1. Creo una lista con los nombres de los archivos y lo los nombres de los dataframes que los van a contener

In [2]:
lista_archivos=['Banco_Mundia_ Ingresos_Per_Capita_segun_encuestas.xls','Banco_Mundial_Acceso_A_Electricidad.xls','Banco_Mundial_Contaminacion_Aire.xls','Banco_Mundial_Crecimiento_del_PBI_%anual.xls','Banco_Mundial_Crecimiento_del_PBI_per_capita.xls','Banco_Mundial_Desempleo_Hombres.xls','Banco_Mundial_Desempleo_mujeres.xls','Banco_Mundial_Gasto_%_PBI.xls','Banco_Mundial_Importaciones_de_Mercaderia.xls','Banco_Mundial_Migration_Rate_Argentina.xls','Banco_Mundial_Personas_Desempleadas_Con_Educacion_Avanzada.xls','Banco_Mundial_Tasa_De_Cambio_Oficial.xls','Banco_Mundial_Tasa_De_Incidencia_De_La_Pobreza.xls','Banco_Mundial_Tasa_de_Mortalidad_en_un_año.xls','Banco_Mundial_Volumenes_internacionales_de_migrantes.xls']
lista_nombres_df=['i_ingresos','i_electricidad','i_contaminacion','i_crecimiento_PBI','i_crecimiento_PBI_capita','i_desempleo_hombres','i_desempleo_mujeres','i_Gasto_PBI','i_importaciones','migration_rate','i_desempleados_educacion','i_tasa_de_cambio','i_tasa_pobreza','i_mortalidad']

In [3]:
dataframes = {}  # Diccionario para almacenar los dataframes

for file, df_name in zip(lista_archivos, lista_nombres_df):
    dataframes[df_name] = pd.read_excel(file,sheet_name='Data',skiprows= 3)  # Leyendo el archivo y guardándolo en el diccionario

In [4]:
dataframes['i_crecimiento_PBI_capita'].head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.635318,-0.709567,2.885166,1.515938,4.917139,4.738330,0.181216,-18.698324,17.225300,NaN
1,NaN,AFE,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,-2.342974,5.093992,2.330796,1.724021,2.436215,...,1.400457,1.186759,0.166840,-0.490259,-0.105886,-0.169822,-0.587557,-5.421662,1.683557,0.842507
2,Afganistán,AFG,Crecimiento del PIB per cápita (% anual),NY.GDP.PCAP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.002522,-0.964803,-1.666416,-0.345802,-0.253594,-1.688577,0.932833,-5.364666,-22.929194,NaN


2. Creo un diccionario con los nombres de los indicadores mas cortos para que sea mas fácil leer los titulos en tablas y gráficos.

In [5]:
titulos_nuevos = [
    "Ingresos",
    "Electricidad",
    "Contaminación",
    "Crecimiento PBI",
    "Crecimiento PBI per cápita",
    "Desempleo Hombre",
    "Desempleo mujeres",
    "Gasto PBI",
    "Importaciones",
    "Migracion neta",
    "Desempleados con educación avanzada",
    "Tasa de cambio",
    "Pobreza",
    "Mortalidad"
]

titulos_viejos = ['Consumo medio o ingresos per cápita según encuestas, población total (USD por día según la PPA de 2011)',
       'Acceso a la electricidad (% de población)',
       'Contaminación del aire por PM2,5, población expuesta a niveles que exceden el valor indicativo de la Organización Mundial de la Salud (OMS) (% del total)',
       'Crecimiento del PIB (% anual)',
       'Crecimiento del PIB per cápita (% anual)',
       'Desempleo, varones (% de participación masculina en la fuerza laboral) (estimación nacional)',
       'Desempleo, mujeres (% de la población activa femenina) (estimación modelado OIT)',
       'Gasto (% del PIB)',
       'Importaciones de mercaderías (US$ a precios actuales)',
       'Migración neta',
       'Personas desempleadas con educación avanzada (% del desempleo total)',
       'Tasa de cambio oficial (UMN por US$, promedio para un período)',
       'Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)',
       'Tasa de mortalidad en un año (por cada 1.000 personas)']

diccionario_titulos = dict(zip(titulos_viejos, titulos_nuevos))

2. Creo función de transformacion que debe aplicarse a cada dataframe creado a partir de cada archivo y guardado en el diccionario dataframes. En esta funcion se pasan los años a las columas, se acortan los nombres de los indicadores y se eliminan columnas que no se necesitan

In [6]:
def transformacion (df, dic):
    
    df= df.drop(['Country Name','Indicator Code'], axis=1)
    df_melted = df.melt(id_vars=['Indicator Name','Country Code'], var_name='Year', value_name=df['Indicator Name'].max())
    df_melted=df_melted.drop(columns=['Indicator Name'])
    df_melted.rename(columns =dic, inplace=True)
    return df_melted

3. Aplico la funcion de transformacion a todos los archios y los guardo en un diccionario de dataframes df_convertidos:

In [7]:
df_convertidos={}

In [8]:
for df_name, df in dataframes.items():
    df_convertidos[df_name]=transformacion(df,diccionario_titulos)

In [9]:
df_convertidos['i_crecimiento_PBI_capita'].sort_values(by=['Country Code','Year'])

,Country Code,Year,Crecimiento PBI per cápita
0,ABW,1960,NaN
266,ABW,1961,NaN
532,ABW,1962,NaN
798,ABW,1963,NaN
1064,ABW,1964,NaN
...,...,...,...
15693,ZWE,2018,2.909395
15959,ZWE,2019,-8.177320
16225,ZWE,2020,-9.670405
16491,ZWE,2021,6.271613


In [11]:
from functools import reduce

# Asegúrate de que melts_1 contenga los DataFrames que quieres fusionar

# Define una función para fusionar dos DataFrames en función de las columnas 'Country Name' y 'Year'
def merge_dataframes(left, right):
    return pd.merge(left, right, on=['Country Code', 'Year'])

# Utiliza reduce para aplicar la función de fusión a todos los DataFrames en la lista
merged_df_modelo = reduce(merge_dataframes, df_convertidos.values())

# Imprime el DataFrame resultante

### dataset que usa Jime

In [12]:
merged_df_modelo

,Country Code,Year,Ingresos,Electricidad,Contaminación,Crecimiento PBI,Crecimiento PBI per cápita,Desempleo Hombre,Desempleo mujeres,Gasto PBI,Importaciones,Migracion neta,Desempleados con educación avanzada,Tasa de cambio,Pobreza,Mortalidad
0,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.594000
1,AFE,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.757511e+09,-90849.0,NaN,NaN,NaN,20.918108
2,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.700000e+07,2606.0,NaN,1.719656e+01,NaN,31.921000
3,AFW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.880000e+09,-847.0,NaN,NaN,NaN,26.363067
4,AGO,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.280000e+08,-43749.0,NaN,2.866844e-08,NaN,26.659000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,XKX,2022,NaN,NaN,NaN,3.511707,4.924753,NaN,NaN,NaN,NaN,NaN,NaN,9.509155e-01,NaN,NaN
16754,YEM,2022,NaN,NaN,NaN,NaN,NaN,NaN,26.299,NaN,5.715000e+09,NaN,NaN,1.110203e+03,NaN,NaN
16755,ZAF,2022,NaN,NaN,NaN,2.042299,1.187663,27.39,31.426,NaN,1.362080e+11,NaN,15.35,1.635585e+01,NaN,NaN
16756,ZMB,2022,NaN,NaN,NaN,4.744942,1.895518,NaN,6.245,NaN,9.047000e+09,NaN,NaN,1.693759e+01,NaN,NaN


In [16]:
merged_df_modelo.to_csv('indicadores_modelo.csv')

### dataset que usa Silvi

In [14]:
merged_df_modelo_melted = pd.melt(merged_df_modelo, id_vars=['Country Code','Year'], 
value_vars=['Ingresos','Electricidad','Contaminación','Crecimiento PBI','Crecimiento PBI per cápita','Desempleo Hombre','Desempleo mujeres','Gasto PBI','Importaciones','Migracion neta','Desempleados con educación avanzada','Tasa de cambio','Pobreza','Mortalidad'], 
var_name='Indicador', value_name='Valor')

In [15]:
merged_df_modelo_melted[merged_df_modelo_melted['Valor'].notna()]

,Country Code,Year,Indicador,Valor
11616,NPL,2003,Ingresos,3.780
11747,COD,2004,Ingresos,1.000
12014,COG,2005,Ingresos,3.390
12177,SEN,2005,Ingresos,3.320
12349,IRQ,2006,Ingresos,10.680
...,...,...,...,...
234341,XKX,2021,Mortalidad,7.220
234342,YEM,2021,Mortalidad,6.845
234343,ZAF,2021,Mortalidad,11.432
234344,ZMB,2021,Mortalidad,6.973


In [17]:
merged_df_modelo_melted.to_csv('indicadores_en_una_columna.csv')